> # Garbage Classification using PyTorch

In this kernel, we'll classify waste into various categories using Residual Networks in PyTorch. The dataset being used: [Garbage_data](https://www.kaggle.com/asdasdasasdas/garbage-classification)



In [ ]:
import os
import torch
import torchvision
from torchvision.datasets.utils import download_url
from torch.utils.data import random_split
import torchvision.models as models
import torch.nn as nn
import torch.nn.functional as F
import time
import copy
import glob
import torchvision.transforms as transforms
import torch.optim as optim

In [ ]:
data_dir = '../input/garbage-classification/garbage classification/Garbage classification'

print(os.listdir(data_dir))
classes = os.listdir(data_dir)
print("\nClasses:", classes)
print("\nNumber of Classes:", len(classes))

## Image Augmentation:

We'll be augmenting images using `torchvision.transforms`, which helps in reducing the chances of overfitting or the inability to generalize on new data.

This kernel uses images of size `128x128` after augmentation due to GPU restrictions, but feel free to switch to a higher resolution if you have access to more resources!

In [ ]:
from torchvision.datasets import ImageFolder
import torchvision.transforms as transforms

In [ ]:
transformations = transforms.Compose([
                        transforms.Resize((128, 128)),
                        transforms.ToTensor(),
                       ])

In [ ]:
dataset = ImageFolder(data_dir, transform=transformations)
print(len(dataset))

In [ ]:
img, label = dataset[0]
print(img.shape, label)
img

## Visualizing Samples:

Here is a helper function for visualizing sample images:

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

def show_example(img, label):
    print('Label: ', dataset.classes[label], "("+str(label)+")")
    plt.imshow(img.permute(1, 2, 0))

Here is the first image of the dataset with its label:

In [ ]:
img, label = dataset[0]
show_example(img, label)

In [ ]:
show_example(*dataset[2000])

## Loading Data:

Let's start by creating training and validation sets. We'll be using 90% of the data for training and the rest for validation.

In [ ]:
random_seed = 42
torch.manual_seed(random_seed);

In [ ]:
len(dataset)

In [ ]:
val_per = 0.1
train_size = len(dataset) - int(val_per*len(dataset))
val_size = int(val_per*len(dataset))


train_ds, val_ds = random_split(dataset, [train_size, val_size])
len(train_ds), len(val_ds)

We can now create data loaders for training and validation, to load the data in batches.

This kernel uses a batch size of 32; you can increase the batch size if you have VRAM <16GB.

In [ ]:
from torch.utils.data.dataloader import DataLoader

batch_size= 32

In [ ]:
train_dl = DataLoader(train_ds, batch_size, shuffle=True, num_workers=4, pin_memory=True)
val_dl = DataLoader(val_ds, batch_size*2, num_workers=4, pin_memory=True)

In [ ]:
from torchvision.utils import make_grid

def show_batch(dl):
    for images, labels in dl:
        fig, ax = plt.subplots(figsize=(12, 6))
        ax.set_xticks([]); ax.set_yticks([])
        ax.imshow(make_grid(images, nrow=16).permute(1, 2, 0))
        break

In [ ]:
show_batch(train_dl)

## Model:

Here is the base of our classification model:

In [ ]:
def accuracy(outputs, labels):
    _, preds = torch.max(outputs, dim=1)
    return torch.tensor(torch.sum(preds == labels).item() / len(preds))

class ImageClassificationBase(nn.Module):
    def training_step(self, batch):
        images, labels = batch 
        out = self(images)                  # Generate predictions
        loss = F.cross_entropy(out, labels) # Calculate loss
        return loss
    
    def validation_step(self, batch):
        images, labels = batch 
        out = self(images)                    # Generate predictions
        loss = F.cross_entropy(out, labels)   # Calculate loss
        acc = accuracy(out, labels)           # Calculate accuracy
        return {'val_loss': loss.detach(), 'val_acc': acc}
        
    def validation_epoch_end(self, outputs):
        batch_losses = [x['val_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()   # Combine losses
        batch_accs = [x['val_acc'] for x in outputs]
        epoch_acc = torch.stack(batch_accs).mean()      # Combine accuracies
        return {'val_loss': epoch_loss.item(), 'val_acc': epoch_acc.item()}
    
    def epoch_end(self, epoch, result):
        print("Epoch [{}], train_loss: {:.4f}, val_loss: {:.4f}, val_acc: {:.4f}".format(
            epoch, result['train_loss'], result['val_loss'], result['val_acc']))

In [ ]:
class ResNet(ImageClassificationBase):
    def __init__(self):
        super().__init__()
        # Use a pretrained model
        self.network = models.resnet50(pretrained=True)
        # Replace last layer
        num_ftrs = self.network.fc.in_features
        self.network.fc = nn.Linear(num_ftrs, 87)
    
    def forward(self, xb):
        return torch.sigmoid(self.network(xb))

In [ ]:
model = ResNet()
model

In [ ]:
for images, labels in train_dl:
    print('images.shape:', images.shape)
    out = model(images)
    print('out.shape:', out.shape)
    print('out[0]:', out[0])
    break

These functions help in working with CUDA. I would personally recommend to train the model on GPUs due to their more processing power and faster training times.

In [ ]:
def get_default_device():
    """Pick GPU if available, else CPU"""
    if torch.cuda.is_available():
        return torch.device('cuda')
    else:
        return torch.device('cpu')
    
def to_device(data, device):
    """Move tensor(s) to chosen device"""
    if isinstance(data, (list,tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)

class DeviceDataLoader():
    """Wrap a dataloader to move data to a device"""
    def __init__(self, dl, device):
        self.dl = dl
        self.device = device
        
    def __iter__(self):
        """Yield a batch of data after moving it to device"""
        for b in self.dl: 
            yield to_device(b, self.device)

    def __len__(self):
        """Number of batches"""
        return len(self.dl)

In [ ]:
device = get_default_device()
device

We can now wrap our training and validation data loaders using `DeviceDataLoader` for automatically transferring batches of data to the GPU (if available), and use `to_device` to move our model to the GPU (if available).

In [ ]:
train_dl = DeviceDataLoader(train_dl, device)
val_dl = DeviceDataLoader(val_dl, device)
to_device(model, device);

In [ ]:
@torch.no_grad()
def evaluate(model, val_loader):
    model.eval()
    outputs = [model.validation_step(batch) for batch in val_loader]
    return model.validation_epoch_end(outputs)

def fit(epochs, lr, model, train_loader, val_loader, opt_func=torch.optim.SGD):
    history = []
    optimizer = opt_func(model.parameters(), lr)
    for epoch in range(epochs):
        # Training Phase 
        model.train()
        train_losses = []
        for batch in train_loader:
            loss = model.training_step(batch)
            train_losses.append(loss)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
        # Validation phase
        result = evaluate(model, val_loader)
        result['train_loss'] = torch.stack(train_losses).mean().item()
        model.epoch_end(epoch, result)
        history.append(result)
    return history

Before we begin training, let's instantiate the model once again and see how it performs on the validation set with the initial set of parameters.

In [ ]:
model = to_device(ResNet(), device)

In [ ]:
evaluate(model, val_dl)

## Training the Model:

In [ ]:
num_epochs = 3
opt_func = torch.optim.Adam
lr = 5.5e-5

Let's train the model now!

In [ ]:
history = fit(num_epochs, lr, model, train_dl, val_dl, opt_func)

We can also plot the valdation set accuracies to study how the model improves over time.

In [ ]:
num_epochs = 25
opt_func = torch.optim.Adam
lr = 6e-6

In [ ]:
history = fit(num_epochs, lr, model, train_dl, val_dl, opt_func)

## Accuracy & Loss Plots:

In [ ]:
def plot_accuracies(history):
    accuracies = [x['val_acc'] for x in history]
    plt.plot(accuracies, '-x')
    plt.xlabel('epoch')
    plt.ylabel('accuracy')
    plt.title('Accuracy vs. No. of epochs');

In [ ]:
plot_accuracies(history)

In [ ]:
def plot_losses(history):
    train_losses = [x.get('train_loss') for x in history]
    val_losses = [x['val_loss'] for x in history]
    plt.plot(train_losses, '-bx')
    plt.plot(val_losses, '-rx')
    plt.xlabel('epoch')
    plt.ylabel('loss')
    plt.legend(['Training', 'Validation'])
    plt.title('Loss vs. No. of epochs');

In [ ]:
plot_losses(history)

## Making Predictions:

In [ ]:
base_path='../input/data-test'
test_dataset = ImageFolder(base_path+'/TEST', transform=transformations)
test_dataset

Let's define a helper function `predict_image`, which returns the predicted label for a single image tensor.

In [ ]:
def predict_image(img, model):
    # Convert to a batch of 1
    xb = to_device(img.unsqueeze(0), device)
    # Get predictions from model
    yb = model(xb)
    # Pick index with highest probability
    _, preds  = torch.max(yb, dim=1)
    # Retrieve the class label
    return dataset.classes[preds[0].item()]

In [ ]:
img, label = test_dataset[44]
plt.imshow(img.permute(1, 2, 0))
print('Label:', dataset.classes[label], ', Predicted:', predict_image(img, model))

In [ ]:
img, label = test_dataset[14 2, 0))
print('Label:', dataset.classes[label], ', Predicted:', predict_image(img, model))

In [ ]:
img, label = test_dataset[560]
plt.imshow(img.permute(1, 2, 0))
print('Label:', dataset.classes[label], ', Predicted:', predict_image(img, model))

In [ ]:
test_loader = DeviceDataLoader(DataLoader(test_dataset, batch_size*2), device)
result = evaluate(model, test_loader)
result

In [ ]:
torch.save(model.state_dict(), 'waste-classification-data.pth')

## Testing on Downloaded Images:

I'll be downloading files using `urllib` as for uploading files in Kaggle, you'll need to create a new dataset (which is not suitable for a few images).

I'll be downloading an image of an plastic bottle:

In [ ]:
import urllib.request
# urllib.request.urlretrieve("data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQABAAD/2wCEAAkGBxIQEBISEhAVFRMWEhIVFRUYFRkYEBgSFxIWFxUVFxcYHCggGBolHxUVITEhJSkrLi4uGB8zODMsNygtLisBCgoKDg0OGhAQGi0lHR0tLS0tLS0vKy0tLS0rLS0tLS0tKzUtLS0tLS0tLS0tLS0tLy0tLS0tLSstNS0tLS0tLf/AABEIAOEA4QMBIgACEQEDEQH/xAAcAAEAAgMBAQEAAAAAAAAAAAAABAUCAwYBBwj/xABBEAACAQICBQkFBAkEAwAAAAAAAQIDEQQhBRIxQVEGIjJhcYGRobETQpLB0RQzUoIHI1NystLh8PEVQ2OiYpPC/8QAGgEBAQEBAQEBAAAAAAAAAAAAAAEDBAIFBv/EACURAQEAAQMDAgcAAAAAAAAAAAABAgMRMQQSQWGRITJRgbHB4f/aAAwDAQACEQMRAD8A+4gAAAAAAAAAAAAAAA47lNy/o4LE/ZvZuVRRjKUpSUKSUrtLWzbfdv2lW/0i6+VJ0ZPglOVu9NI+efpOw9SOmqrqzjK+o4XitSFPVWqtV5Sas83vK7SFSPs1bEub/DrasV+VZI+z0/RaeenMrHzeo6jPHPaV9jwHKqvUzaopZ+7L5zLHC8qbyhGcIvXqqknCV2pOMpJuLWzm8XtPhnJ6phv9+MN+2q/ky5wOCw9WVP2FKi6v2uCgliJQbj7Ob2wvKPOUecllkXU6HTmNuzxp9Tqd8lv4ffkz0xp7EZHxX1QAAAAAAAAAAAAAAAAAAAAAAAAAAAAAMKtRRi5SaSSbbeSSW1t7kZmNSKaaaTTVmnmmntTQH5q/SDj8PW0vUrRxPt4SUGpK+pGNsoRss0rbVtuyPjMbQdO0KM28s7Sa82W3LzBx/wBXrfZqfMXs+dTs4ZU46yyyvfWy4mOLo4mVOyU/hpr1Z+j6XOTSnxj5HVzfU/qDoLSFKK5+HqNdUJfIl0cdhXUhz3Rl7eD19WXtIwSk7p5u+sodb2bGyRoH29LpKfcofzEzD4iClJVaTlrV6ElrJbE5Rlfckte+3OyPWpqTsrPSxnfL+323RGkKWIowqUqiqQa6VrNtZO6aTi+KaViaR8DTpxglSjFQ2rVtq9qsSD80+2AAAAAAAAAAAAAAAAAAAAAAAAAAAAADOT5XaUbvh6cmrr9ZJOzS/Anub3/5L7TGN9jScl0nlH957+7N9xxFOnd3ebbu+LZphj5Z55bfCK6noiH4EtmzLsNj0bHZn4l3DCStsNdSi1tRv31hdOKb/TI8DGeiofhW1PPis0XGoSKGBlPdZdY76TTjdya0hqPUfR3rh1o6+LujhK+GlSkmtu1PcdVoXF+0gv7s96MM55b4XwsgAZtAAAAAAAAAAAAAAAAAAAAAAAAAAAcrykr69VR3QVu9q78rGrR2G3vuNeIlrSnN75SfjLIssHF6qVjbiMeayqZIp8c25W6kXs6DKXHU2pu/V6E3WxrwdNOWZf4dZFJo/p9x0GHhkS1ZBRs0+Hmt6I9OHscQ1Ho1Frx4X3pE6UbETG9CnLfCol+V/wCURdtlwmemFF81GZ4ewAAAAAAAAAAAAAAAAAAAAAAAA8lsZ6eS2Acj7ByjGK2tr+/IusPC2XDLzK2gvu8t8fmWlPa+1+prkyjOaKPSizfb8i9kyj0rtfavQkWo2B6a7/Q6Wic1genHv9DpaJKsbZEHFfdVO2H8ROkytxdW0XHfKUPKT+hItXFDZ3s2Guhs72bDy9AAAAAAAAAAAAAAAAAAAAAAAAAAA52C1HZ+48+u3DxuSFUd7q9nn0XfMqMRJyUpN5tu/eT8Hh42VuHFmtZJU6rtsl8JUaQnuale981YuJ0uwptJJp7rZEhWjC5yWre50FJztsfgUGESc0mjoKOGjbZ6irGVSdT8PoQVHWnHWTv7uVk2s9tsyfUwsH7vm/qQMRJQnBRjZtrO7fvWas7okWr3DLmq+/M2mrDPLvNp4ewAAAAAAAAAAAAAAAAAAAAAAAAMADk8W4wnOO1Rll9OvgKFdS2T1erV+mRhiY86plnry/jJWGrJJayjHLYuj6s28MfLOUHb71fA/qV2NunbWUu5r5lvLFU7bYlRpGpFvm2ezYQasNnJZJdef1LmjH/mfg/qUuFlacb8ToqWIh+NeJKsaqi/5fJkeEbVIq6z2PevFfMnVcTC2U/AgQm/aRWs5Q2vW2X7OOwKvsHG0Fnfj2m80YJ3gn2+pvM2gAAAAAAAAAAAAAAAAAAAAAAAAGABx2la961SSWx6r/LzW2Y4F1JpNpSW68rZeB5i4/rK379Ttzk7ZGzBfq4q7Wzdf6G/hheU32U7dCPx/wBCtx7knZ5djLT7XC3S/wCsvoVOkqyk8r7tzXqiK14d85bX1XLuhRf7JfGvoUGFmteOds0dNRrw/EiVY11KD/Zr4/6EWEEqiTas8tRNSze/NJpeJPrYmFukiqpZ11L3U737uskWunwjvBeHhkbiNo93pp8b+FySZtAAAAAAAAAAAAAAAAAAAAAAAAAMBgcPpmvrV6kkui9X4MrkfA03PPV2/wDkl5NG/GQtWrX/AGk8t9nK6fhma8NKMLXns4LPzN5wwvKxjhJJb1+aL+RX45Shvfl8kWH26Fsm34fUq9I1tbdw3oiscPUvJJ3fVdXLmlhb/wC3L4o/U5/D1Frx7UdPhsRHivij9SUjXUwtvca7ZxNNGmlJK6knla8ZNdexWS7yVjMRFp5p96fpcqNGJqpFtWSvn3MRXY4Gd4Lqy8MiQRNGfdrrba7LksyrUAAAAAAAAAAAAAAAAAAAAAAAAAAHB6brOpiajt0ZandBtEHDOUtsH8S9GTcdTccRWu0n7Sbs96bbT80Q3aDupTe+1svK5vOGF5WFPBXV+d/1+pExsHT2PxSv5G2lpBJZJd8pfykXG13P8Pc3cDyjVbaT8krltQwt/dqeEfmylw750ct6OooV7W2eZKsRqmFkt0126i/+jyhTSks1NO17OLkuuy2LxI+lKutK1k+FmzDRbcZ3lktVgdjo6d6a6ub3LJEohaI+6T4ttdhNMq2AAQAAAAAAAAAAAAAAAAAAAAAAAAfP9PVnUxNRtdF6itlzYt/V+JDpRk81HLds+ckbsbzq9Z7va1M/zNEapGd06UW09tr27rLM6JwwvKWqE+v4V8pGvFU5U9tn3f1PIrEW6El2xt6mnEa22TV+GVyDKjUcmlbwV2WNLC32OX/rZV4eb1o24o6HWnGOSvwsKRpWDmvxW/dS9ZmVOklJXtJXzs1rLuTtYgVKNZu8oSfYmydgqUkrtNJWu2nZN7EB1GiqmtTXBNxXYthMK7QeUJLhN+iLExvLaAAIoAAAAAAAAAAAAAAAAAAAAABgAfPtJ/qcRVhZSvJyT25T51muKuaZYmo1lNrgr5eYxl5Vqz3+2qZ/ne08pU1G2vVjd9UvXI6Jw57yxWKrbNa/dH6Gqs5S6Xp9EWEaFG2VR36pW9SDirRdot/Ff5IDGhBxadnk7lutJzStC6fFopqMm2ld+Ni3pYOG2Tt21F/KRXk8ZXltqPxS9DKnK2drvjtfizZOlRjsqNvhdy9Ej2VRZat00+GT7ec33ZAdFoGFqN98m32breRZFXoCprRnlZa+S/Ki0Mby2nAACKAAAAAAAAAAAAAAAAAAAAAAYAHznTqX2qrqZR1udfZr++11XNcaMZJc66avuyJHKej7PGy4TUZpbrtar84vxIGAkqcpJJWudOPyxzZfMsYaLp2vrPxXyNGJwUYp2Za0aya6ESu0mrp2SWQFfGm/xIso4dPbJsqcNDNXSeZ0eGrQS6C8voBFnh45ZvLsMVd7ONtu8nV8WksoRX99SRErYyUopO1ls5qyfVlcDqeTaSoW95Set+9t9Gi0Kzk5S1cPFvbK833vLysWZz3l0TgABFAAAAAAAAAAAAAAAAAAAAAAAAcjy9w33NRLY5QfetZfwvxOX9rGMm5SfYkn53PpmksDGvTlCV7O2a6Sa2NdZ8/0ryHxXO9nKFRN5Z6s/CWXmdGlljttXPq4Zb7xUYrlhSoXTs3w1syg0h+kC6doRS2XbbfgkZ6U5AYxtuWEm3vcWm2uHMbyOdxXIarFtzwuJ+Co/C0TqxmHpfux32539k2ly3qKWym1ddXzOl0by4hOylC3X7vjsOGoci6rd40sV2eyn/JkXmj+QFdu/wBlxD43U4xa4PJJ956ymn6e5v8ATf2d7S0rTqrmteOfgSNVytGO1tJdrdkVmiORGKj0aEad98pRS8I3Z2+guTSoWlUnrzWxWtBPjxZy55YzitMccr4XtCkoRjFbEkl2JWNgByuoAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAB4egAAAAAAAAAAAAAAAAAAAB//9k==", "plastic.jpg")
urllib.request.urlretrieve("data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQABAAD/2wCEAAkGBxIQEBISEhAVFRMWEhIVFRUYFRkYEBgSFxIWFxUVFxcYHCggGBolHxUVITEhJSkrLi4uGB8zODMsNygtLisBCgoKDg0OGhAQGi0lHR0tLS0tLS0vKy0tLS0rLS0tLS0tKzUtLS0tLS0tLS0tLS0tLy0tLS0tLSstNS0tLS0tLf/AABEIAOEA4QMBIgACEQEDEQH/xAAcAAEAAgMBAQEAAAAAAAAAAAAABAUCAwYBBwj/xABBEAACAQICBQkFBAkEAwAAAAAAAQIDEQQhBRIxQVEGIjJhcYGRobETQpLB0RQzUoIHI1NystLh8PEVQ2OiYpPC/8QAGgEBAQEBAQEBAAAAAAAAAAAAAAEDBAIFBv/EACURAQEAAQMDAgcAAAAAAAAAAAABAgMRMQQSQWGRITJRgbHB4f/aAAwDAQACEQMRAD8A+4gAAAAAAAAAAAAAAA47lNy/o4LE/ZvZuVRRjKUpSUKSUrtLWzbfdv2lW/0i6+VJ0ZPglOVu9NI+efpOw9SOmqrqzjK+o4XitSFPVWqtV5Sas83vK7SFSPs1bEub/DrasV+VZI+z0/RaeenMrHzeo6jPHPaV9jwHKqvUzaopZ+7L5zLHC8qbyhGcIvXqqknCV2pOMpJuLWzm8XtPhnJ6phv9+MN+2q/ky5wOCw9WVP2FKi6v2uCgliJQbj7Ob2wvKPOUecllkXU6HTmNuzxp9Tqd8lv4ffkz0xp7EZHxX1QAAAAAAAAAAAAAAAAAAAAAAAAAAAAAMKtRRi5SaSSbbeSSW1t7kZmNSKaaaTTVmnmmntTQH5q/SDj8PW0vUrRxPt4SUGpK+pGNsoRss0rbVtuyPjMbQdO0KM28s7Sa82W3LzBx/wBXrfZqfMXs+dTs4ZU46yyyvfWy4mOLo4mVOyU/hpr1Z+j6XOTSnxj5HVzfU/qDoLSFKK5+HqNdUJfIl0cdhXUhz3Rl7eD19WXtIwSk7p5u+sodb2bGyRoH29LpKfcofzEzD4iClJVaTlrV6ElrJbE5Rlfckte+3OyPWpqTsrPSxnfL+323RGkKWIowqUqiqQa6VrNtZO6aTi+KaViaR8DTpxglSjFQ2rVtq9qsSD80+2AAAAAAAAAAAAAAAAAAAAAAAAAAAAADOT5XaUbvh6cmrr9ZJOzS/Anub3/5L7TGN9jScl0nlH957+7N9xxFOnd3ebbu+LZphj5Z55bfCK6noiH4EtmzLsNj0bHZn4l3DCStsNdSi1tRv31hdOKb/TI8DGeiofhW1PPis0XGoSKGBlPdZdY76TTjdya0hqPUfR3rh1o6+LujhK+GlSkmtu1PcdVoXF+0gv7s96MM55b4XwsgAZtAAAAAAAAAAAAAAAAAAAAAAAAAAAcrykr69VR3QVu9q78rGrR2G3vuNeIlrSnN75SfjLIssHF6qVjbiMeayqZIp8c25W6kXs6DKXHU2pu/V6E3WxrwdNOWZf4dZFJo/p9x0GHhkS1ZBRs0+Hmt6I9OHscQ1Ho1Frx4X3pE6UbETG9CnLfCol+V/wCURdtlwmemFF81GZ4ewAAAAAAAAAAAAAAAAAAAAAAAA8lsZ6eS2Acj7ByjGK2tr+/IusPC2XDLzK2gvu8t8fmWlPa+1+prkyjOaKPSizfb8i9kyj0rtfavQkWo2B6a7/Q6Wic1genHv9DpaJKsbZEHFfdVO2H8ROkytxdW0XHfKUPKT+hItXFDZ3s2Guhs72bDy9AAAAAAAAAAAAAAAAAAAAAAAAAAA52C1HZ+48+u3DxuSFUd7q9nn0XfMqMRJyUpN5tu/eT8Hh42VuHFmtZJU6rtsl8JUaQnuale981YuJ0uwptJJp7rZEhWjC5yWre50FJztsfgUGESc0mjoKOGjbZ6irGVSdT8PoQVHWnHWTv7uVk2s9tsyfUwsH7vm/qQMRJQnBRjZtrO7fvWas7okWr3DLmq+/M2mrDPLvNp4ewAAAAAAAAAAAAAAAAAAAAAAAAMADk8W4wnOO1Rll9OvgKFdS2T1erV+mRhiY86plnry/jJWGrJJayjHLYuj6s28MfLOUHb71fA/qV2NunbWUu5r5lvLFU7bYlRpGpFvm2ezYQasNnJZJdef1LmjH/mfg/qUuFlacb8ToqWIh+NeJKsaqi/5fJkeEbVIq6z2PevFfMnVcTC2U/AgQm/aRWs5Q2vW2X7OOwKvsHG0Fnfj2m80YJ3gn2+pvM2gAAAAAAAAAAAAAAAAAAAAAAAAGABx2la961SSWx6r/LzW2Y4F1JpNpSW68rZeB5i4/rK379Ttzk7ZGzBfq4q7Wzdf6G/hheU32U7dCPx/wBCtx7knZ5djLT7XC3S/wCsvoVOkqyk8r7tzXqiK14d85bX1XLuhRf7JfGvoUGFmteOds0dNRrw/EiVY11KD/Zr4/6EWEEqiTas8tRNSze/NJpeJPrYmFukiqpZ11L3U737uskWunwjvBeHhkbiNo93pp8b+FySZtAAAAAAAAAAAAAAAAAAAAAAAAAMBgcPpmvrV6kkui9X4MrkfA03PPV2/wDkl5NG/GQtWrX/AGk8t9nK6fhma8NKMLXns4LPzN5wwvKxjhJJb1+aL+RX45Shvfl8kWH26Fsm34fUq9I1tbdw3oiscPUvJJ3fVdXLmlhb/wC3L4o/U5/D1Frx7UdPhsRHivij9SUjXUwtvca7ZxNNGmlJK6knla8ZNdexWS7yVjMRFp5p96fpcqNGJqpFtWSvn3MRXY4Gd4Lqy8MiQRNGfdrrba7LksyrUAAAAAAAAAAAAAAAAAAAAAAAAAAHB6brOpiajt0ZandBtEHDOUtsH8S9GTcdTccRWu0n7Sbs96bbT80Q3aDupTe+1svK5vOGF5WFPBXV+d/1+pExsHT2PxSv5G2lpBJZJd8pfykXG13P8Pc3cDyjVbaT8krltQwt/dqeEfmylw750ct6OooV7W2eZKsRqmFkt0126i/+jyhTSks1NO17OLkuuy2LxI+lKutK1k+FmzDRbcZ3lktVgdjo6d6a6ub3LJEohaI+6T4ttdhNMq2AAQAAAAAAAAAAAAAAAAAAAAAAAAfP9PVnUxNRtdF6itlzYt/V+JDpRk81HLds+ckbsbzq9Z7va1M/zNEapGd06UW09tr27rLM6JwwvKWqE+v4V8pGvFU5U9tn3f1PIrEW6El2xt6mnEa22TV+GVyDKjUcmlbwV2WNLC32OX/rZV4eb1o24o6HWnGOSvwsKRpWDmvxW/dS9ZmVOklJXtJXzs1rLuTtYgVKNZu8oSfYmydgqUkrtNJWu2nZN7EB1GiqmtTXBNxXYthMK7QeUJLhN+iLExvLaAAIoAAAAAAAAAAAAAAAAAAAAABgAfPtJ/qcRVhZSvJyT25T51muKuaZYmo1lNrgr5eYxl5Vqz3+2qZ/ne08pU1G2vVjd9UvXI6Jw57yxWKrbNa/dH6Gqs5S6Xp9EWEaFG2VR36pW9SDirRdot/Ff5IDGhBxadnk7lutJzStC6fFopqMm2ld+Ni3pYOG2Tt21F/KRXk8ZXltqPxS9DKnK2drvjtfizZOlRjsqNvhdy9Ej2VRZat00+GT7ec33ZAdFoGFqN98m32breRZFXoCprRnlZa+S/Ki0Mby2nAACKAAAAAAAAAAAAAAAAAAAAAAYAHznTqX2qrqZR1udfZr++11XNcaMZJc66avuyJHKej7PGy4TUZpbrtar84vxIGAkqcpJJWudOPyxzZfMsYaLp2vrPxXyNGJwUYp2Za0aya6ESu0mrp2SWQFfGm/xIso4dPbJsqcNDNXSeZ0eGrQS6C8voBFnh45ZvLsMVd7ONtu8nV8WksoRX99SRErYyUopO1ls5qyfVlcDqeTaSoW95Set+9t9Gi0Kzk5S1cPFvbK833vLysWZz3l0TgABFAAAAAAAAAAAAAAAAAAAAAAAAcjy9w33NRLY5QfetZfwvxOX9rGMm5SfYkn53PpmksDGvTlCV7O2a6Sa2NdZ8/0ryHxXO9nKFRN5Z6s/CWXmdGlljttXPq4Zb7xUYrlhSoXTs3w1syg0h+kC6doRS2XbbfgkZ6U5AYxtuWEm3vcWm2uHMbyOdxXIarFtzwuJ+Co/C0TqxmHpfux32539k2ly3qKWym1ddXzOl0by4hOylC3X7vjsOGoci6rd40sV2eyn/JkXmj+QFdu/wBlxD43U4xa4PJJ956ymn6e5v8ATf2d7S0rTqrmteOfgSNVytGO1tJdrdkVmiORGKj0aEad98pRS8I3Z2+guTSoWlUnrzWxWtBPjxZy55YzitMccr4XtCkoRjFbEkl2JWNgByuoAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAB4egAAAAAAAAAAAAAAAAAAAB//9k=", "plastic.jpg")

Let us load the pretrained model:

In [ ]:
loaded_model = model
loaded_model.load_state_dict(torch.load('./waste-classification-data.pth'))

Now, we'll open the image and predict:

In [ ]:
from PIL import Image
from pathlib import Path
image = Image.open(Path('./plastic.jpg'))

example_image = transformations(image)
plt.imshow(example_image.permute(1, 2, 0))
predict_image(example_image, loaded_model)

In [ ]:
!pip install jovian --upgrade

In [ ]:
import jovian

In [ ]:
jovian.commit(project='garbage-classification')